### 1. GeoMap coordenadas de posiciones

Obtiene en el csv con nombre coordinates-60-200-10-0.csv, las coordenadas de las posiciones del json instance-60-200-10-0.json.

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Leer el archivo JSON
with open('instances/instance-60-200-10-0.json', 'r') as f:
    data = json.load(f)

# Extraer las distancias y los identificadores de las posiciones
distances = []
position_ids = set()
for distance in data['distances']:
    distances.append(distance['value'])
    position_ids.add(distance['source'])
    position_ids.add(distance['target'])

# Convertir el conjunto de identificadores en una lista
position_ids = list(position_ids)

# Crear un diccionario para mapear los identificadores de posición a índices
position_id_to_index = {pos_id: idx for idx, pos_id in enumerate(position_ids)}

# Crear una matriz de distancias
num_positions = len(position_ids)
distance_matrix = np.zeros((num_positions, num_positions))
for distance in data['distances']:
    source_idx = position_id_to_index[distance['source']]
    target_idx = position_id_to_index[distance['target']]
    distance_matrix[source_idx, target_idx] = distance['value']
    distance_matrix[target_idx, source_idx] = distance['value']

# Función de pérdida para minimizar la diferencia entre las distancias reales y las distancias euclidianas
def loss_function(coordinates, distance_matrix):
    n = len(coordinates) // 2
    coordinates = coordinates.reshape(-1, 2)
    loss = 0
    for i in range(n):
        for j in range(i+1, n):
            euclidean_distance = np.linalg.norm(coordinates[i] - coordinates[j])
            loss += (euclidean_distance - distance_matrix[i, j])**2
    return loss

# Realizar múltiples iteraciones para ajustar las coordenadas
num_iterations = 5
for _ in range(num_iterations):
    # Inicializar las coordenadas aleatoriamente para cada iteración
    initial_coordinates = np.random.rand(num_positions * 2)

    # Minimizar la función de pérdida para ajustar las coordenadas
    result = minimize(loss_function, initial_coordinates, args=(distance_matrix,), method='L-BFGS-B')

    # Obtener las coordenadas ajustadas
    coordinates = result.x.reshape(-1, 2)

    # Actualizar las coordenadas iniciales para la siguiente iteración
    initial_coordinates = result.x

# Crear un DataFrame para almacenar las coordenadas
coordinates_df = pd.DataFrame(coordinates, columns=['x', 'y'], index=position_ids)

# Normalizar las coordenadas al rango (0, 1)
coordinates_df['x'] = (coordinates_df['x'] - coordinates_df['x'].min()) / (coordinates_df['x'].max() - coordinates_df['x'].min()) * 2
coordinates_df['y'] = (coordinates_df['y'] - coordinates_df['y'].min()) / (coordinates_df['y'].max() - coordinates_df['y'].min())

# Generar un archivo CSV con las coordenadas
coordinates_df.to_csv('coordinates-60-200-10-0.csv')

# Mostrar la gráfica de posiciones
plt.figure(figsize=(8, 8))
for idx, row in coordinates_df.iterrows():
    plt.plot(row['x'], row['y'], 'bo')
    plt.text(row['x'], row['y'], idx, fontsize=12, ha='center', va='bottom')
plt.title('Posiciones')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid(True)
plt.xlim(0, 2)
plt.ylim(0, 1)
plt.show()


### 2. Trayectoria de vehículos (Dashboard Solución)

Obtiene un csv con la lista de movimientos de cada vehículo mostrando la posición.

In [ ]:
import json
import csv

# Leer el archivo JSON
with open('solutionsComplete/solutionComplete-60-200-10-0.json', 'r') as file:
    data = json.load(file)

# Iterar sobre cada ruta
for i, route in enumerate(data['routes']):
    # Crear el nombre del archivo CSV
    csv_filename = f'geomap-vehicle-{i+1}.csv'
    
    # Extraer las posiciones actuales y eliminar duplicados manteniendo el orden
    seen = set()
    current_positions = []
    for desplazamiento in route['trayectoria']['desplazamientos']:
        pos = desplazamiento.get('currentPos')
        if pos is not None and pos not in seen:
            seen.add(pos)
            current_positions.append(pos)
    
    # Escribir en el archivo CSV
    with open(csv_filename, 'w', newline='') as csvfile:
        fieldnames = ['node']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for pos in current_positions:
            writer.writerow({'node': pos})

print("Archivos CSV generados con éxito.")


### 3. Trayectoria de vehículos (Dashboard Solución)

Añade las coordenadas.

In [ ]:
import json
import csv

# Leer el archivo JSON
with open('solutionsComplete/solutionComplete-60-200-10-0.json', 'r') as file:
    data = json.load(file)

# Leer el archivo coordinates.csv
coordinates = {}
with open('coordinates-60-200-10-0.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        node = int(row['node'])  # Convertir a entero si es necesario
        coordinates[node] = {'latitude': row['latitude'], 'longitude': row['longitude']}

# Iterar sobre cada ruta
for i, route in enumerate(data['routes']):
    # Crear el nombre del archivo CSV
    csv_filename = f'geomap-vehicle-{i+1}.csv'
    
    # Extraer las posiciones actuales y eliminar duplicados manteniendo el orden
    seen = set()
    current_positions = []
    for desplazamiento in route['trayectoria']['desplazamientos']:
        pos = desplazamiento.get('currentPos')
        if pos is not None and pos not in seen:
            seen.add(pos)
            current_positions.append(pos)
    
    # Escribir en el archivo CSV
    with open(csv_filename, 'w', newline='') as csvfile:
        fieldnames = ['node', 'latitude', 'longitude']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for pos in current_positions:
            coord = coordinates.get(pos, {'latitude': 'N/A', 'longitude': 'N/A'})  # Manejar nodos sin coordenadas
            writer.writerow({
                'node': pos,
                'latitude': coord['latitude'],
                'longitude': coord['longitude']
            })

print("Archivos CSV generados con éxito.")


### 4. Trabajos (Dashboard Instancia)

Obtiene un csv con la lista de posiciones de los trabajos. Las filas impares muestran la posición inciial de los trabajos y las pares la posición final.

In [ ]:
import json
import pandas as pd

# Leer el archivo JSON
with open('instances/instance-60-200-10-0.json', 'r') as file:
    data = json.load(file)

# Extraer el array de jobs
jobs = data['jobs']

# Crear una lista para almacenar los datos procesados
processed_data = []

# Procesar cada job en el array
for job in jobs:
    
    # Añadir filas para initialPos y endPos
    processed_data.append({'node': job['initialPos']})
    processed_data.append({'node': job['endPos']})

# Convertir la lista de datos procesados a un DataFrame de pandas
df = pd.DataFrame(processed_data)

# Guardar los datos en un archivo CSV
df.to_csv('jobs-coordinates-1.csv', index=False)


### 5. Trabajos (Dashboard Instancia)

Añade las coordenadas.

In [ ]:
import pandas as pd

# Leer el archivo jobs-coordinates-1.csv
jobs_df = pd.read_csv('jobs-coordinates-1.csv')

# Leer el archivo coordinates-60-200-10-0.csv
coordinates_df = pd.read_csv('geomap/geomap-60-200-10-0/coordinates-60-200-10-0.csv')

# Agregar una columna de índice para preservar el orden original
jobs_df['index'] = jobs_df.index

# Realizar la combinación en base a la columna 'node'
merged_df = pd.merge(jobs_df, coordinates_df, on='node', suffixes=('_job', '_coord'))

# Reordenar los datos de acuerdo con el índice original
merged_df = merged_df.sort_values(by='index').drop(columns='index')

# Guardar el DataFrame resultante en un nuevo archivo CSV
merged_df.to_csv('jobs-coordinates-1.csv', index=False)


### 6. Trabajos (Dashboard Instancia)

Crea en un json las queries de forma que cada bloque corresponde a una query del GeoMap. En el GeoMap se represetan las queries como trabajos individuales de forma que cada query tiene la posición inicial de un trabajo y la posición final.

#### 6.1 Añadir los csvContent al GeoMap

In [ ]:
import pandas as pd
import json

# Leer el archivo CSV
jobs_df = pd.read_csv('jobs-coordinates-1.csv')

# Función para convertir un número en una letra de columna de Excel
def number_to_excel_column(n):
    result = ''
    while n > 0:
        n, remainder = divmod(n - 1, 26)
        result = chr(65 + remainder) + result
    return result

# Inicializar variables
new_blocks = []
ref_id_index = 3  # Comienza desde 'C'

# Procesar el archivo CSV en bloques de 3 filas
for i in range(0, len(jobs_df), 2):
    # Seleccionar 3 filas
    chunk = jobs_df.iloc[i:i + 2]
    
    # Crear el CSV
    csv_content = "node,latitude,longitude\r\n" + "\r\n".join([f"{row.node},{row.latitude},{row.longitude}" for row in chunk.itertuples()])
    
    # Crear el bloque csvContent
    ref_id = number_to_excel_column(ref_id_index)
    new_block = {
        "csvContent": csv_content,
        "datasource": {
            "type": "grafana-testdata-datasource",
            "uid": "edksd0i6rrdhcd"
        },
        "refId": ref_id,
        "scenarioId": "csv_content"
    }
    new_blocks.append(new_block)
    
    # Incrementar el índice de ref_id
    ref_id_index += 1

# Guardar los nuevos bloques en un archivo JSON
with open('instance-60-200-10-0-geomap2.json', 'w') as file:
    json.dump(new_blocks, file, indent=2)


#### 6.2 Añadir las rutas de los trabajos leyendo los csvContent

In [ ]:
import pandas as pd
import json

# Leer el archivo CSV
jobs_df = pd.read_csv('jobs-coordinates-1.csv')

# Función para convertir un número en una letra de columna de Excel
def number_to_excel_column(n):
    result = ''
    while n > 0:
        n, remainder = divmod(n - 1, 26)
        result = chr(65 + remainder) + result
    return result

# Inicializar variables
layer_blocks = []
ref_id_index = 5  # Comienza desde 'C'

# Procesar el archivo CSV en bloques de 3 filas
for i in range(0, len(jobs_df), 2):
    # Seleccionar 3 filas
    chunk = jobs_df.iloc[i:i + 2]
    
    # Crear el bloque config
    ref_id = number_to_excel_column(ref_id_index)
    layer_block = {
        "config": {
            "arrow": 0,
            "style": {
                "color": {
                    "fixed": "light-orange"
                },
                "lineWidth": 2,
                "opacity": 1,
                "rotation": {
                    "fixed": 0,
                    "max": 360,
                    "min": -360,
                    "mode": "mod"
                },
                "size": {
                    "fixed": 5,
                    "max": 15,
                    "min": 2
                },
                "symbol": {
                    "fixed": "img/icons/marker/circle.svg",
                    "mode": "fixed"
                },
                "symbolAlign": {
                    "horizontal": "center",
                    "vertical": "center"
                },
                "textConfig": {
                    "fontSize": 12,
                    "offsetX": 0,
                    "offsetY": 0,
                    "textAlign": "center",
                    "textBaseline": "middle"
                }
            }
        },
        "filterData": {
            "id": "byRefId",
            "options": ref_id
        },
        "location": {
            "mode": "auto"
        },
        "name": f"Layer {ref_id_index - 1}",
        "opacity": 0.7,
        "tooltip": True,
        "type": "route"
    }
    layer_blocks.append(layer_block)
    
    # Incrementar el índice de ref_id
    ref_id_index += 1

# Guardar los nuevos bloques en un archivo JSON
with open('instance-60-200-10-0-geomap3.json', 'w') as file:
    json.dump(layer_blocks, file, indent=2)
